In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import re
import csv
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report,precision_recall_fscore_support
# import ipwhois

In [2]:
def split_len(x):
    return len(x.split('.'))

In [336]:
df_train_benin = pd.read_csv('Data/Train_Benign_Traffic.csv',encoding="latin1")
df_train_mal = pd.read_csv('Data/Train_Detections.csv')
df_test = pd.read_csv('Data/Test_Set_Hackathon.csv')
df_train_ips = pd.read_csv('Data/Train_IPS.csv')
df_ips = pd.read_csv('Data/IPTypes.csv').set_index('ip')

In [337]:
features = df_test.columns[1:]
# easter_eggs = df[df.dst_ip.map(split_len) > 4 ].dst_ip
df_train_benin['label'] = 'benign'
df_train = pd.concat([df_train_benin,df_train_mal]).reset_index()
df_train = df_train.drop(['index','Unnamed: 0','app_name','app_risk'],axis=1)
df_train.protocol = df_train.protocol.replace({'UDP':2,'TCP':1})
df_train = df_train[df_train.dst_ip.map(split_len) < 5 ].reset_index()
df_train = df_train[df_train.src_ip.map(split_len) < 5 ].reset_index().drop(['index','level_0'],axis=1)
df_train.head()

/Users/jeremybensoussan/anaconda3/envs/ITC/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


,client,dst_ip,dst_port,epoch_time,label,protocol,received_bytes,sent_bytes,src_ip
0,2,222.58.210.46,443.0,1547242292,benign,1,44096,67712,10.197.88.33
1,5,110.244.46.16,443.0,1547468725,benign,1,7712,1990,10.0.11.69
2,2,23.130.21.5,443.0,1547415821,benign,1,12928,8464,192.168.174.137
3,2,71.55.163.78,443.0,1547152973,benign,1,19808,4376,192.168.190.183
4,5,230.0.0.1,NaN,1547070189,benign,1,0,252,10.0.11.2


In [338]:
df_train = df_train.join(df_ips,on='src_ip',rsuffix='_src').drop('Unnamed: 0',axis=1)
df_train = df_train.join(df_ips,on='src_ip',rsuffix='_src').drop('Unnamed: 0',axis=1)
df_train = df_train.join(df_ips,on='dst_ip',lsuffix='_dst').drop(['type', 'subtype', 'Unnamed: 0'],axis=1)
df_train['log_sent_bytes'] = np.log1p(df_train.sent_bytes).astype(int)
df_train['log_received_bytes'] = np.log1p(df_train.received_bytes).astype(int)
df_train =  pd.concat([pd.DataFrame(df_train.dst_ip.apply(lambda x: x.split('.')).values.tolist(),
             columns=['dst_int_1','dst_int_2','dst_int_3','dst_int_4']),df_train],axis=1)
df_train =  pd.concat([pd.DataFrame(df_train.src_ip.apply(lambda x: x.split('.')).values.tolist(),
             columns=['src_int_1','src_int_2','src_int_3','src_int_4']),df_train],axis=1)

In [339]:
df_train.head()

,src_int_1,src_int_2,src_int_3,src_int_4,dst_int_1,dst_int_2,dst_int_3,dst_int_4,client,dst_ip,...,protocol,received_bytes,sent_bytes,src_ip,type_dst,subtype_dst,type_src,subtype_src,log_sent_bytes,log_received_bytes
0,10,197,88,33,222,58,210,46,2,222.58.210.46,...,1,44096,67712,10.197.88.33,Internal,Class A,Internal,Class A,11,10
1,10,0,11,69,110,244,46,16,5,110.244.46.16,...,1,7712,1990,10.0.11.69,Internal,Class A,Internal,Class A,7,8
2,192,168,174,137,23,130,21,5,2,23.130.21.5,...,1,12928,8464,192.168.174.137,Internal,Class C,Internal,Class C,9,9
3,192,168,190,183,71,55,163,78,2,71.55.163.78,...,1,19808,4376,192.168.190.183,Internal,Class C,Internal,Class C,8,9
4,10,0,11,2,230,0,0,1,5,230.0.0.1,...,1,0,252,10.0.11.2,Internal,Class A,Internal,Class A,5,0


In [467]:
suspicious_ips = df_train[(df_train['label'] == 'Malware')].dst_ip
df_train_benign = df_train[df_train['label'] == 'benign']
suspicious_ips = suspicious_ips[~suspicious_ips.isin(df_train_benign.dst_ip)]
suspicious_ips[suspicious_ips != '8.8.8.8']

50439     192.168.98.222
50440    212.189.218.209
50447        179.239.5.6
50450      114.77.190.35
50457       58.59.105.78
50458    212.189.218.209
50460       73.135.105.9
50462     192.168.98.222
50476     184.79.251.128
50479      173.99.238.81
50485    204.134.112.241
50496      110.18.47.145
50498      60.88.164.186
50504     40.201.191.165
50506      222.58.204.12
50507     192.168.98.222
50510    149.204.181.113
50515       28.71.63.178
50516    212.189.218.209
50518     219.105.49.212
50519     200.187.98.169
50526      114.77.190.35
50529        97.184.0.10
50532     60.169.245.138
50536        97.184.0.10
50544       40.231.9.114
50550    168.210.195.122
50558     40.241.253.240
50559      114.77.190.35
50564     192.168.98.222
              ...       
50856     110.237.145.10
50862      93.248.214.84
50872     211.185.208.80
50877      204.46.87.247
50884        97.184.0.10
50885        19.32.66.61
50888      24.196.83.205
50889      114.77.190.35
50895      114.77.190.35


In [ ]:
def plot_count(series):
    counts = series.value_counts()
    plt.figure(figsize=(5,2))
    sns.barplot(counts.index,counts)

In [ ]:
def plot_pca(df):
    pca = PCA(n_components=2)
    to_pca = df.drop(['dst_ip','src_ip'],axis=1)
    to_pca = to_pca.dropna()
    labels_pca = to_pca.label
    proj = pca.fit_transform(to_pca.drop(['label'],axis=1))
    plt.figure(figsize=(10,6))
    sns.scatterplot(proj[:,0],proj[:,1],hue=labels_pca)

In [ ]:
#dummies 
df_train = pd.concat([df_train,pd.get_dummies(df_train.dst_port,prefix='port')],axis=1)

In [ ]:
#separate df for client

df_1 = df_train[df_train.client == 1].drop('client',axis=1)
df_2 = df_train[df_train.client == 2].drop('client',axis=1)
df_3 = df_train[df_train.client == 3].drop('client',axis=1)
df_4 = df_train[df_train.client == 4].drop('client',axis=1)
df_5 = df_train[df_train.client == 5].drop('client',axis=1)

In [ ]:
list_of_df = [df_1,df_2,df_3,df_4,df_5]

for i,df in enumerate(list_of_df):
    plot_count(df.label)
    plt.title(f'client {i+1}')
    plt.show()

In [ ]:
for i, df in enumerate(list_of_df):
    plot_pca(df)
    plt.title(f'client {i+1}')
    plt.show()

In [ ]:
prediction = []
models = []
scores = []
for df in list_of_df:
    X = df.drop(['label','src_ip','dst_ip'],axis=1)
    y = df.label
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    rf = RandomForestClassifier(class_weight='balanced')
    rf.fit(X_train,y_train)
    p = rf.predict(X_test)
    prediction.append(p)
    scores.append(precision_recall_fscore_support(y_test,p))
    models.append(rf)
    

In [720]:
# preds = df_test.apply(lambda x: True if x.dst_ip in suspicious_ips.to_list()
#                       and x.sent_bytes != 1282
#                       and x.received_bytes != 4176
#                       and x.received_bytes != 2002944
#                       else False, axis=1).astype(bool)

pred_1 = df_test.apply(lambda x: True 
                      if 
                      x.dst_ip in ['10.200.105.200', '10.200.105.201', '10.200.105.202']
                      and x.dst_port == 53
                      and (x.received_bytes > 0 or x.sent_bytes > 0)
                      and x.received_bytes < 200
                      else False, axis=1).astype(bool)

pred_2 = df_test.apply(lambda x: True 
                      if 
                      x.dst_port == 53
                      and x.received_bytes == 0 
                      and x.sent_bytes == 0
                      else False, axis=1).astype(bool)

pred_3 = df_test.apply(lambda x: True 
                      if 
                      x.dst_ip == '8.8.8.8'
                      and x.protocol == 'UDP'
                      else False, axis=1).astype(bool)

pred_4 = df_test.apply(lambda x: True if x.dst_ip in suspicious_ips.to_list()
                       and x.dst_ip != '8.8.8.8'
                      and x.sent_bytes != 1282
                       and x.client == 2
                       and x.dst_port == 80
                      else False, axis=1).astype(bool)

pred_5 = df_test.apply(lambda x: True if 
                       x.dst_port == 53
                       and x.dst_ip == '10.200.105.201'
                       and x.received_bytes <= 540
                      else False, axis=1).astype(bool)

pred_6 = df_test.apply(lambda x: True if 
                       x.dst_ip == '10.200.105.200'
                       and x.received_bytes <= 200
                      else False, axis=1).astype(bool)

print(pred_4.value_counts())
df_test[pred_1 | pred_2 | pred_3 | pred_4 | pred_5 | pred_6].shape

False    11145
True        13
dtype: int64


(164, 8)

In [705]:
preds = {v:bool(k) == True for v,k in dict(preds).items()}

In [706]:
r = requests.post('http://192.168.1.24:30000/submission',
                   json={ 'team':'DeepZuchinis',
                          'password':'4lbertKmus',
                          'submission': preds
                         },
                   verify=False)

In [707]:
r.text

"Your precision is so bad that we don't even want to display it..."

In [174]:
df_train[df_train.dst_ip.isin(suspicious_ips)].label.value_counts()

Malware    292
Adware      11
Name: label, dtype: int64